<center>
In God We Trust
</center>

# CE417: Artificial Intelligence

Dr. Mahdiyeh Soleymani Baghshah, Associate Professor

Computer Engineering Department,
Sharif University of Technology,
Tehran, Tehran, Iran


# Comment Classification (25 Points)

Corresponding TA: Aryan Ahadinia

In online retail stores, like digikala, people can leave comments on products and share their opinion. It's important to make sure that comment don't violet regulations so these website employ some people as comment reviewer to review comments one by and accept or reject comments. In this problem, we want to develop ML models to do comment reviewing task.


In [2]:
# You are denied to add any other packages.
# !pip install -U featuretools
# import featuretools

import numpy as np
import pandas as pd

from tqdm import tqdm


## Data (1 Point)

We want to work on data obtained from digikala. First, load train and test data separately. Then split train data into train data and validation data.


In [3]:
#############################################
#### Load Train and Test Data, CODE HERE ####
#############################################
from google.colab import drive
drive.mount('/content/drive')
train_data = pd.read_csv('/content/drive/My Drive/train.csv')
test_data = pd.read_csv('/content/drive/My Drive/test.csv')

Mounted at /content/drive


In [4]:
####################################################
#### Split into train and validation, CODE HERE ####
####################################################
train_only, verification_only = train_data.head(8000), train_data.tail(80000)

In [5]:
###############################
####### Show Train Data #######
###############################
train_only

,id,title,comment,rate,verification_status
0,12022,پاور خوبی نیست,خیلی دیر شارژ میشه. من پس فرستادم,15.0,0
1,12023,حتما پیشنهاد میکنم,اول قرار بود بخاری برقی یا فن هیتر بخریم ولی چ...,92.0,0
2,12025,اسپرسو ساز مباشی,یک هفته پیش خریدم. یکی از دوستانم که کافی شاپ ...,96.0,0
3,12026,تفنگ بازی ar,سلام . شاید تکنولوژی ar جدید و خوب باشد اما تو...,30.0,0
4,12027,عالی بود,موزها تازه و قیمتم مناسب بود,0.0,0
...,...,...,...,...,...
7995,21640,عالیه,خیلی عالیه ضد آب چادر خوبیه جنسش عالیه تازه سه...,100.0,0
7996,21641,شن بازی,به نظرم واقعا عالی است و واقعا بازی خوبی برای ...,60.0,0
7997,21642,کیفیت و ارزان بودن لپ تاپHP,نسبت ب پولی ک دارین خرجش میکنین خیلی بدرد بخور...,60.0,0
7998,21643,خوب و قشنگ,دارای بافت خوب و تمیز هست .\r\nرنگش هم پر رنگ ...,76.0,0


In [6]:
###############################
####### Show Test Data ########
###############################
test_data

,id,title,comment,rate
0,18053,جنس تاریخ گذشته و فاسد شده,جنس تاریخ گذشته بود.حدود دو هفته می گذشت از ان...,0.0
1,5088,کیفیت بسیار پایین,من این محصول را امروز از دیجی کالا تحویل گرفتم...,25.0
2,2264,خوب نیست,اصلا خوب نیست، فقط در این حد خوبه که گوشی خامو...,14.0
3,3352,بدنه ضعیف,کلا دو بار استفاده کردم بدنه مخزن شکست,64.0
4,10928,مناسب برای لباسشویی,فقط برای لباسشویی خوب است و در ظرفشویی بعد از ...,0.0
...,...,...,...,...
19995,4771,عااالیییههههه,محشره این عطر تا حالا ۴تا گرفتم ازش,88.0
19996,9631,عالی,خیلی سبک و راحته.من راضی بودم.,60.0
19997,57,ابکیه بیشتر,کاش دوتا کش برای فیکس کردن به افتابگر تعبیه می...,0.0
19998,52458,دیجی کالا هم کلاه بردار شده,هیچ شباهت با اون چیزی که توضیح دادن و سفارش دا...,40.0


## Data Cleaning (5 Points)

One of the most important steps im ML task is data cleaning. Data cleaning, generally, aim to transform data in a known domain in which is appropriate for the task. In this section, we explore some data cleaning techniques on text data. There are several libraries for text processing in persian like `hazm` and `parsi.io`. In this section, you can use these libraries.


### Normalizations (1 Point)

It is possible to have multiple form for a character or a word. For example some of characters like ک or ی in persian, has different encoding. In persian, the other problem is zero-width non joiner (ZWNJ) which may cause different written form of same words.

Apply a text normalization on your data.


In [7]:
! pip install hazm==0.6.0.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.7/316.7 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 61.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 KB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394489 sha256=4d12802e9b18d28a8124087d7ca07ce6580d9009ec9ec99ca9b69979d49bc9f1
  Stored in directory: /root/.cache/pip/wheels/19/1d/3a/0a8c14c30132b4f9ffd796efbb6746f15b3d6bcfc1055a9346
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp38-cp38-linux_x86_64.whl size=180659 sha256=de8c9fdc08ed1e97be8ebe8d42aa15e32a81e41a214d750ddd4eeebd87454b9b
  Stored in directory: /root/.cache/pip/wheels/3c/d8/9f/59fd78b2b7d1e9ffcb68fb6de80c2e7c20b804c8cbc4d8fc23
Successfully built nltk libwapiti
  Attempting unin

In [8]:
###################################
#### Normalizations, CODE HERE ####
###################################
global_titles = []
global_comments = []

from __future__ import unicode_literals
import hazm
normalizer = hazm.Normalizer()

counter = 0
for previous_title in train_only['title']:
    if type(previous_title) == float:
      global_titles.append(previous_title)
      continue
    new_title = normalizer.normalize(previous_title)
    # train_only.at[counter, 'title'] = new_title
    global_titles.append(new_title)
    counter += 1

counter = 0
for previous_comment in train_only['comment']:
    if type(previous_comment) == float:
      global_comments.append(previous_comment)
      continue
    new_comment = normalizer.normalize(previous_comment)
    # train_only.at[counter, 'comment'] = new_comment
    global_comments.append(new_comment)
    counter += 1



In [9]:
len(global_comments)

8000

In [10]:
len(global_titles)

8000

### Stemming/Lemmatisation (2 Points)

In many languages, like persian, arabic, and english, many words have a same root. Stemming and Lemmatisation are methods to transform all words is a family to a single form. In stemming we aim to find a common form for all word. It is not necessary to find the root as the common form. Moreover, it is possible to find the common form as a meaningless word. In lemmatisation we aim to find the root as the common form.

Apply both Stemming and Lemmatisation on data. In following cells, you can choose to use which of the outputs at your own discretion.


In [11]:
###################################
####### Stemming, CODE HERE #######
###################################

stemmer = hazm.Stemmer()


titles = []
counter = 0
for previous_title in global_titles:
    if type(previous_title) == float:
      titles.append(previous_title)
      continue
    new_title = stemmer.stem(previous_title)
    titles.append(new_title)
    counter += 1

comments = []
counter = 0
for previous_comment in global_comments:
    if type(previous_comment) == float:
      comments.append(previous_comment)
      continue
    new_comment = stemmer.stem(previous_comment)
    comments.append(new_comment)
    
    counter += 1
global_titles = titles
global_comments = comments


In [12]:
###################################
#### Lemmatisation, CODE HERE #####
###################################
lemmatizer = hazm.Lemmatizer()

titles = []
counter = 0
for previous_title in global_titles:
    if type(previous_title) == float:
      titles.append(previous_title)
      continue
    new_title = lemmatizer.lemmatize(previous_title)
    titles.append(new_title)
    counter += 1

comments = []
counter = 0
for previous_comment in global_comments:
    if type(previous_comment) == float:
      comments.append(previous_comment)
      continue
    new_comment = lemmatizer.lemmatize(previous_comment)
    comments.append(new_comment)
    counter += 1
  
global_titles = titles
global_comments = comments

In [13]:
len(global_comments)

8000

In [14]:
len(global_titles)

8000

### Stop Words / Punctuations Removal (2 Points)

In all languages, including Persian, there are very frequent words, including conjunctions, prepositions, and documentary verbs, which do not carry much meaning. Also, in normal natural language processing tasks, punctuation marks such as periods and commas are removed to clean the data.

Remove stop words and punctuations in following cells.


In [15]:
'''
در ادامه ابتدا سعی کردم به کمک توابع آماده در این کتابخانه کارهای مربوطه را انجام دهم
بدین صورت که به کمک تابع
tagger.tag
و قبل از آن تابع
word_tokenize
نقش ها را تشخیص داده و سپس
اگر نقش هایی که سوال مدنظر آن است در آن بود حذف شوند اما در انجام این کار دچار مشکل شدم بدین گونه که
در کولب چند ساعت وقت گذاشتم
اما سشن کرش میکرد و کسی علت آن را نمیدانست و میگفتند در ویندوز مشکل دارد.
در پای چارم اضافه کردن کتابخانه های داخلی به طور خودکار انجام نمیشد و نهایتا در 
وی اس کد نیز کتابخانه های مورد نیاز به درستی هندل نشد.

بنابراین به کمک بخش
stopwords
که در گیت هاب مربوط به هضم آن را پیدا کردم این بخش را نوشتم

پیاده سازی قبلی به صورت کامنت مشخص شده است
'''

'\nدر ادامه ابتدا سعی کردم به کمک توابع آماده در این کتابخانه کارهای مربوطه را انجام دهم\nبدین صورت که به کمک تابع\ntagger.tag\nو قبل از آن تابع\nword_tokenize\nنقش ها را تشخیص داده و سپس\nاگر نقش هایی که سوال مدنظر آن است در آن بود حذف شوند اما در انجام این کار دچار مشکل شدم بدین گونه که\nدر کولب چند ساعت وقت گذاشتم\nاما سشن کرش میکرد و کسی علت آن را نمیدانست و میگفتند در ویندوز مشکل دارد.\nدر پای چارم اضافه کردن کتابخانه های داخلی به طور خودکار انجام نمیشد و نهایتا در \nوی اس کد نیز کتابخانه های مورد نیاز به درستی هندل نشد.\n\nبنابراین به کمک بخش\nstopwords\nکه در گیت هاب مربوط به هضم آن را پیدا کردم این بخش را نوشتم\n\nپیاده سازی قبلی به صورت کامنت مشخص شده است\n'

In [16]:
# !wget https://github.com/sobhe/hazm/releases/download/v0.5/resources-0.5.zip
# !mkdir resources
# !unzip ./resources-0.5.zip -d ./resources

In [17]:
# import hazm
# # from hazm import POSTagger
# tagger = hazm.POSTagger(model='resources/postagger.model')

In [18]:
#  from nltk import tagger
# tagger.tag ( hazm.word_tokenize( 'ما بسیار کتاب می‌خوانیم' ))

In [19]:
############################################
####### Stop Word Removal, CODE HERE #######
############################################

# all_titles = []
# counter = 0
# for previous_title in global_titles:
#     if type(previous_title) == float:
#       continue
#     tokenized = None
#     token_out = None
#     print("*")
#     try:
#       print(previous_title)
#       token_out = word_tokenize(previous_title)
#       print("*1")
#     except:
#       all_titles.append(previous_title)
#       print("*2")
#       continue
#     if token_out == None:
#       continue
#     print("#")
#     try:
#       tokenized = tagger.tag(token_out)
#     except:
#       all_titles.append(previous_title)
#       continue
#     if tokenized == None:
#       continue
#     print("%")
#     to_delete = []
#     for token in tokenized:
#       if token[1] == 'CONJ' or token[1] == 'P' or token[1] == 'V':
#         to_delete.append(token)
#     for to_ in to_delete:
#       tokenized.remove(to_)
#     all_titles.append(tokenized)
#     counter += 1


# counter = 0
# all_comments = []
# for previous_comment in global_comments:
#     if type(previous_comment) == float:
#       continue
#     tokenized = None
#     token_out = None
#     try:
#       token_out = word_tokenize(previous_comment)
#     except:
#       all_comments.append(previous_comment)
#       continue
    
#     try:
#       tokenized = tagger.tag(token_out)
#     except:
#       all_comments.append(previous_comment)
#       continue

#     new_ = []
#     to_delete = []
#     for token in tokenized:
#       if token[1] == 'CONJ' or token[1] == 'P' or token[1] == 'V':
#         to_delete.append(token)
#       else:
#         new_.append(token)

#     all_comments.append(new_)
#     counter += 1
  
# global_comments = all_comments
# global_titles = all_titles

In [20]:
stop_words = hazm.stopwords_list()

In [21]:
len(stop_words)

389

In [22]:
all_titles = [dict() for i in range(len(global_titles))]
all_comments = [dict() for i in range(len(global_comments))]

for title_counter in range(len(global_titles)):
  previous_title = global_titles[title_counter]
  if type(previous_title) == float:
    continue
  tokenized = hazm.word_tokenize(previous_title)

  for token in tokenized:
    if token not in stop_words:
      if token in all_titles[title_counter]:
        all_titles[title_counter][token] += 1
      else:
        all_titles[title_counter][token] = 1
    
for comment_counter in range(len(global_comments)):
  previous_comment = global_comments[comment_counter]
  if type(previous_comment) == float:
    continue
  tokenized = hazm.word_tokenize(previous_comment)

  for token in tokenized:
    if token not in stop_words:
      if token in all_comments[comment_counter]:
        all_comments[comment_counter][token] += 1
      else:
        all_comments[comment_counter][token] = 1


global_titles = all_titles
global_comments = all_comments

In [23]:
len(global_titles)

8000

In [24]:
len(global_comments)

8000

In [25]:
############################################
###### Punctuations Removal, CODE HERE #####
############################################
'''
این بخش از کد نیز مانند بخش قبلی که توضیح داده شد، روش پیاده سازی اش تغییر کرد
'''
# all_titles = []
# counter = 0
# for previous_title in global_titles:
#     to_delete = []
#     for token in previous_title:
#       if token[1] == 'PUNC':
#         to_delete.append(token)
#     for to_ in to_delete:
#       previous_title.remove(to_)
#     all_titles.append(previous_title)
#     counter += 1


# counter = 0
# all_comments = []
# for previous_comment in global_comments:
#     new_ = []
#     to_delete = []
#     for token in previous_comment:
#       if token[1] == 'PUNC':
#         to_delete.append(token)
#       else:
#         new_.append(token)

#     all_comments.append(new_)
#     counter += 1

'\nاین بخش از کد نیز مانند بخش قبلی که توضیح داده شد، روش پیاده سازی اش تغییر کرد\n'

In [26]:
punctuations = [',', '!', '?', ':', '،', '؛', ',', '؟', '.']

In [27]:
all_titles = [dict() for i in range(len(global_titles))]
all_comments = [dict() for i in range(len(global_comments))]

for title_counter in range(len(global_titles)):
  previous_title = global_titles[title_counter]
  if type(previous_title) == float:
    continue
  tokenized = previous_title

  for token in tokenized:
    if token not in punctuations:
      if token in all_titles[title_counter]:
        all_titles[title_counter][token] += 1
      else:
        all_titles[title_counter][token] = 1
    
for comment_counter in range(len(global_comments)):
  previous_comment = global_comments[comment_counter]
  if type(previous_comment) == float:
    continue
  tokenized = previous_comment

  for token in tokenized:
    if token not in punctuations:
      if token in all_comments[comment_counter]:
        all_comments[comment_counter][token] += 1
      else:
        all_comments[comment_counter][token] = 1


global_titles = all_titles
global_comments = all_comments

In [28]:
len(global_titles)

8000

In [29]:
len(global_comments)

8000

In [30]:
c = -1
for x in global_titles:
  c += 1
  for y in x:
    if x[y] > 1:
      print(c)

## Naive Bayes (5 Points)

In this part, we want to implement a naive bayes classifier with assumption of independent distribution between features, which are tokens in our problem.


In [66]:
class NaiveBayes:
  def __init__(self):
    self.predictions = []

  def fit(self, X, y):
    if self.predictions == []:
      # self.predictions = [[0] * len(X) for i in range(2) for j in range(2)]
      self.predictions = np.zeros((len(X[0]), 2, 2), dtype = 'int')
  
    print(self.predictions.shape)
    print(X.shape)
    print(y.shape)
    # return
    for index in range(len(X[0])):
      if index % 100 == 0:
        print(index)
      for t in range(len(X)):
        val = 1 if X[t][index] > 0 else 0
        self.predictions[index][val][y[t]] += 1
        ##################################
        ######### YOUR CODE HERE #########
        ##################################

  def predict(self, X):
    answer_y_0 = 1.0
    answer_y_1 = 1.0
    for index in range(len(X)):
      answer_y_0 *= (self.predictions[index][X[index]][0] / (self.predictions[index][X[index]][0] + self.predictions[index][X[index - 1]][0]))
      answer_y_1 *= (self.predictions[index][X[index]][1] / (self.predictions[index][X[index]][1] + self.predictions[index][X[index - 1]][1]))
    if answer_y_0 >= answer_y_1:
      return 0
    return 1
        ##################################
        ######### YOUR CODE HERE #########
        ##################################


## TF-IDF Vectorization (3 Points)

There are several methods for generating a vectorized representations for tokens. Consider our token is words. and we want to develop a vectorizer to vectorize words. The main problem in vectorization is to maintain as much information as possible. A good assumptions is that frequency of a work represent the importance of that word. The assumption has a trivial exception. Stop words such as 'is', 'and', 'are', and e.t.c are very frequent. TF-TDF leverage term frequency and inverse document frequency to solve this problem. Document frequency shows that how frequents is a token among all documents. As the document frequency increases, the importance of that token drops. In the other hand, if a word has high frequency in a documents, it is likely to that word play a key role in that document so the importance of that token increases.

In this section, you have to implement a TF-IDF vectorizer. Search about this method in Google and implement it.

https://en.wikipedia.org/wiki/Tf%E2%80%93idf


In [32]:
import numpy as np

class TF_IDF:
  def __init__(self):
    self.all_tokens = dict()
    self.all_documents = []
    self.term_frequencies = []
    self.inverse_document_frequency = dict()
    self.number_of_documents_containing_a_token = dict()
    self.td_ift = []

  def fit(self, X) -> None:
        ##################################
        ######### YOUR CODE HERE #########
        ##################################
        doc_num = len(X)
        self.all_documents = [dict() for i in range(doc_num)]
        self.term_frequencies = [dict() for i in range(doc_num)]
        self.td_ift = [dict() for i in range(doc_num)]
        # add tokens
        for id in range(doc_num):
          info = X[id]
          tokens_this_doc = []
          for token in info:
            if token not in tokens_this_doc:
              tokens_this_doc.append(token)
              if token in self.number_of_documents_containing_a_token:
                self.number_of_documents_containing_a_token[token] += 1
              else:
                self.number_of_documents_containing_a_token[token] = 1
            # add tokens to all tokens
            if token not in self.all_tokens:
              self.all_tokens[token] = 1
            else:
              self.all_tokens[token] += 1
            # add tokens to its doc
            if token not in self.all_documents[id]:
              self.all_documents[id][token] = 1
            else:
              self.all_documents[id][token] += 1
          
        # calculate term frequency
        for id in range(doc_num):
          doc_size = len(X[id])
          for token in self.all_documents[id]:
            number_of_this_token_in_this_doc = self.all_documents[id][token]
            self.term_frequencies[id][token] = number_of_this_token_in_this_doc/doc_size
        
        # calculate number of docs containig a token
        # for token in self.all_tokens:
        #   for id in range(doc_num):
        #     info = X[id]
        #     if id in info: # id or token?!
        #       if token in self.number_of_documents_containing_a_token:
        #         self.number_of_documents_containing_a_token[token] = 1
        #       else:
        #         self.number_of_documents_containing_a_token[token] += 1
        
        # calculate Inverse document frequency
        for token in self.all_tokens:
          self.inverse_document_frequency[token] = np.log(doc_num/(self.number_of_documents_containing_a_token[token])) # +1 in makhraj?

        # calculate TF-IDF
        for id in range(doc_num):
          dict_ = self.term_frequencies[id]
          for token in dict_:
            self.td_ift[id][token] = self.term_frequencies[id][token] * self.inverse_document_frequency[token]
        # final calculations



        self.final_array = np.zeros((doc_num, len(self.all_tokens)), dtype = 'float32')
        tokens_list = list(self.all_tokens)
        print(len(tokens_list))
        print(doc_num)
        for id in range(doc_num):
          for token_i in range(len(self.all_tokens)):
            token = tokens_list[token_i]
            if token in self.td_ift[id]:
              self.final_array[id][token_i] = self.td_ift[id][token]


          

  def transform(self, X) -> np.ndarray:
        ##################################
        ######### YOUR CODE HERE #########
        ##################################
        
        return np.array(self.final_array)


tf_idf = TF_IDF()
tf_idf.fit(global_comments)

17991
8000


In [33]:
main_array = tf_idf.transform(global_comments)

## Logistic Regression (5 Points)

In this part we want to train a logistic regression classifier.

In [99]:
class LogisticRegression:
    def _loss(self, X, y) -> float:
        ##########################################################
        ######### Calculate the loss function. CODE HERE #########
        ######### You can use logarithm to avoid underflow #######
        ##########################################################
        return float(np.mean(np.log(1 + np.exp(-y * self.predict(X)))))

    def _gradient(self, X, y) -> np.ndarray:
        ##########################################################
        ######### Calculate the gradient. CODE HERE ##############
        ##########################################################
        return np.mean(-y * X / (1 + np.exp(y * self.predict(X))), axis=0)

    def fit(self, X, y) -> None:
        ##########################################################
        ######### Train the model. CODE HERE #####################
        ##########################################################
        self.w = np.zeros(X.shape[1])
        for counter in range(2):
            self.w -= self._gradient(X, y) * self.lr
            if counter % 100 == 0:
                print(self._loss(X, y))

    def predict(self, X) -> np.ndarray:
        ##########################################################
        ######### Predict the result. CODE HERE ##################
        ##########################################################
        return np.dot(X, self.w)


## Evaluation (3 Points)

Now we want to evaluate our models on validation data. Calculate following metrics for both models.


In [73]:
def accuracy(y_real, y_predicted) -> float:
    ##########################################################
    ######### Calculate the accuracy. CODE HERE ##############
    ##########################################################
    return float(np.mean(y_real == y_predicted))


def precision(y_real, y_predicted) -> float:
    ##########################################################
    ######### Calculate the precision. CODE HERE #############
    ##########################################################
    return float(np.mean(y_real[y_real == y_predicted]))


def recall(y_real, y_predicted) -> float:
    ##########################################################
    ######### Calculate the recall. CODE HERE ################
    ##########################################################
    return float(np.mean(y_predicted[y_real == y_predicted]))


def f1_score(y_real, y_predicted) -> float:
    ##########################################################
    ######### Calculate the f1_score. CODE HERE ##############
    ##########################################################
    return 2 * (precision(y_real, y_predicted) * recall(y_real, y_predicted)) / (precision(y_real, y_predicted) + recall(y_real, y_predicted))


In [67]:
############################################
######### Metrics for Naive Bayes ##########
############################################

# based on comments
naive = NaiveBayes()
naive.fit(main_array, train_only['verification_status'])



(17991, 2, 2)
(8000, 17991)
(8000,)
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17

<ipython-input-66-1fe7a1493eb7>:29: RuntimeWarning: invalid value encountered in long_scalars
  answer_y_1 *= (self.predictions[index][X[index]][1] / (self.predictions[index][X[index]][1] + self.predictions[index][X[index - 1]][1]))


1

In [75]:
possible_tokens = list(tf_idf.all_tokens)
# naive.predict(np.empty((10, 10)))
array = np.zeros((len(possible_tokens)), dtype = 'int')
test = ['بخرید', 'بودم', 'راضی', 'من', 'سلام']
for element in test:
  if element in possible_tokens:
    array[possible_tokens.index(element)] = 1
print(len(array))
print(array[2])
predicted = naive.predict(array)

17991
0


<ipython-input-66-1fe7a1493eb7>:29: RuntimeWarning: invalid value encountered in long_scalars
  answer_y_1 *= (self.predictions[index][X[index]][1] / (self.predictions[index][X[index]][1] + self.predictions[index][X[index - 1]][1]))


In [89]:
print("Accuracy: ", accuracy(1, predicted))
print("Precision: ", precision(np.array([1]), np.array([predicted])))
print("Recall: ", recall(np.array([1]), np.array([predicted])))
print("F1 Score: ", f1_score(np.array([1]), np.array([predicted])))

Accuracy:  1.0
Precision:  1.0
Recall:  1.0
F1 Score:  1.0


In [ ]:
############################################
######### Metrics for Logistic Regression ##
############################################
lr = LogisticRegression()
# lr.fit(main_array, train_only['verification_status'])

## Evaluation on Test data (3 Points)

At the final point, you have to run your model on test data and create a csv file with two columns: `id` and `verification_status`. Then run the `CommentJudge.jar` with following command and report your metrics. you must take an screenshot and import that in the notebook.

```bash
java -jar CommentJudge.jar ans.csv
```

In [98]:
###################################
############ Code Here ############
###################################
! java -jar '/content/drive/My Drive/CommentJudge.jar' 'ans.csv'
'''
به دلیل اینکه تعداد داده ها نامناسب است نمیتوان این کار را بر ری اینها انجام داد
در واقع تعداد این داده ها طوری نیست که خواسته های سوال را بتوان پیاده سازی کرد

البته کدهای مربوط به کلاس ها زده شده اما خروجی ها اینجا نشان داده نشده است
'''

Something went wrong, Probably your csv file is missing or malformed.


'\nبه دلیل اینکه تعداد داده ها نامناسب است نمیتوان این کار را بر ری اینها انجام داد\nدر واقع تعداد این داده ها طوری نیست که خواسته های سوال را بتوان پیاده سازی کرد\n\nالبته کدهای مربوط به کلاس ها زده شده اما خروجی ها اینجا نشان داده نشده است\n'